In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import seaborn as sns
import os
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def write_input(path, df, id_job, n_cutoffs, algo, dev, n_res, n_seeds, target_key):
    """
    Creates the two input files necessary to run SGD using realkd:
    i) a .json file with calculation details, named "id_job.json", and
    ii) a .xarf file with the data set, named "id_job.xarf".
    Arguments: path(str): path to the folder where the files 
                                   will be written
               df(data frame): data set containing the values for the 
                               candidate descriptive parameters and for
                               the target for all adsorption sites
               id_job(str): job name
               n_cutoffs(int): number of cutoffs to be used in k-Means
                               clustering to generate the propositions
               algo(str): SG search algorithm:
                          PMM_SAMPLER 
                          EMM_SAMPLER
                          EXCEPTIONAL_SUBGROUP_BESTFIRST_BRANCHANDBOUND
                          
                          PMM_SAMPLER uses (std(SG)-std(P))/std(P) as utility function
                          whereas EMM_SAMPLER/EXCEPTIONAL_SUBGROUP_BESTFIRST_BRANCHANDBOUND 
                          use the function specified in dev
               
               dev(str): deviation measure when using EMM_SAMPLER: 
                         cumulative_jensen_shannon_divergence
                         normalized_positive_mean_shift
                         normalized_negative_mean_shift
                         normalized_positive_median_shift
                         normalized_negative_median_shift
                         
               n_res(int): number of results, i.e., number of top-ranked
                           SGs to display
               n_seeds(int): number of seeds to use for the SG search
               target_key(str): label of the variable to be used as target quantity in SGD
    """
    df.to_csv(path+'/'+id_job+'.csv')
    with open(path+'/'+id_job+'.csv', 'r') as file_in:
        data = file_in.read().splitlines(True)
        
    file_out = open(path+'/'+id_job+'.xarf', 'w')
    file_out.write('@relation '+id_job+'\n')
    file_out.write('@attribute sites name\n')
    for variable in list(df.columns):
        file_out.write('@attribute '+variable+' numeric\n')
    file_out.write("@data\n")
    file_out.close()

    with open(path+'/'+id_job+'.xarf', 'a') as file_out:
        file_out.writelines(data[1:])
        file_out.close()
    
    input_file = {}
    input_file = {"type" : "productWorkScheme",
                  "id" : id_job,
                  "workspaces" : [ {
                                "type" : "workspaceFromXarf",
                                "id" : id_job,
                                "datafile" : id_job+".xarf",
                                "propScheme": {"type": "standardPropScheme",
                                                "defaultMetricRule": {"type": "kmeansPropRule",
                                                                       "numberOfCutoffs": n_cutoffs,
                                                                       "maxNumberOfIterations": 1000}}} ],
                    "computations" : [ {
                                "type" : "legacyComputation",
                                "id" : "subgroup_analysis",
                                "algorithm" : algo,
                                "parameters" : {
                                    "dev_measure": dev,
                                    "attr_filter" : "[]",
                                    "cov_weight" : "1.0",
                                    "num_res" : n_res,
                                    "num_seeds" : n_seeds,
                                    "targets" : "["+target_key+"]"
                                             }
                  }],
                  "computationTimeLimit" : 3600000
                     }
    with open(path+'/'+id_job+'.json','w') as outfile:
        json.dump(input_file, outfile, indent=4)
        

def analyze(file_results):
    """
    Extracts information about the identified SGs from the realkd output file into a data frame.
    The values of relative SG size (coverage), utility function, quality function, the mean value of
    the target property in the SG as well as the SG constraints are considered for each identified SG. 
    Argument: file_results: realkd's output json file
    """
    list_coverages=[]
    list_utility_function=[]
    list_quality_function=[]
    list_target_mean=[]
    list_constraints=[]
    
    with open(file_results) as json_file:
        data = json.load(json_file)
        for index in range(len(data)):
            coverage=data[index].get('measurements')[0].get('value')
            utility_function=data[index].get('measurements')[1].get('value')
            quality_function=coverage*utility_function
            target_mean=data[index].get('descriptor').get('targetLocalModel').get('means')
            list_attributes=data[index].get('descriptor').get('selector').get('attributes')
            list_operators=[]
            list_cutoffs=[]
            constraints=[]
            for i in list(range(0,len(list_attributes))):
                list_operators.append(data[index].get('descriptor').get('selector').get('constraints')[i].get('type'))
                list_cutoffs.append(round(data[index].get('descriptor').get('selector').get('constraints')[i].get('value'),4))

            list_operators = [op.replace('lessOrEquals', '<=') for op in list_operators]
            list_operators = [op.replace('greaterOrEquals', '>=') for op in list_operators]
            list_operators = [op.replace('lessThan', '<') for op in list_operators]
            list_operators = [op.replace('greaterThan', '>') for op in list_operators]
    
            for i in list(range(0,len(list_attributes))):
                if i == 0:
                    constraints=list_attributes[0]+list_operators[0]+str(list_cutoffs[0])
                else:
                    constraints=constraints+' & '+list_attributes[i]+list_operators[i]+str(list_cutoffs[i])
            list_coverages.append(coverage)
            list_utility_function.append(utility_function)
            list_quality_function.append(quality_function)
            list_target_mean.append(*target_mean)
            list_constraints.append(constraints)
            
    df = pd.DataFrame(list(zip(list_coverages,
                               list_utility_function,
                               list_quality_function,
                               list_target_mean,
                               list_constraints)), 
                      columns =['coverage','utility','quality','target_mean','constraints'])
    return(df)


def get_pareto_frontier(Xs, Ys, maxX=True, maxY=True):
    """
    Identifies the Pareto front.
    Arguments: Xs (array): values of first objective
               Ys (array): values of second objective
    """
    sorted_list = sorted([[Xs[i], Ys[i], i] for i in range(len(Xs))], reverse=maxY)
    pareto_front = [sorted_list[0]]
    for pair in sorted_list[1:]:
        if maxY:
            if pair[1] >= pareto_front[-1][1]:
                pareto_front.append(pair)
        else:
            if pair[1] <= pareto_front[-1][1]:
                pareto_front.append(pair)
    
    return(pareto_front)

def get_pareto_region(Xs, Ys, threshold, maxX=True, maxY=True):
    """
    Identifies the Pareto region defined as the solutions at the Pareto front plus the solutions within 
    a threhold distance to the solutions of the Pareto front.
    Arguments: Xs (array): values of first objective
               Ys (array): values of second objective
               threshold (float): threshold distance to the Pareto front
    """
    sorted_list = sorted([[Xs[i], Ys[i], i] for i in range(len(Xs))], reverse=maxY)
    pareto_front = [sorted_list[0]]
    for pair in sorted_list[1:]:
        if maxY:
            if pair[1] >= pareto_front[-1][1]:
                pareto_front.append(pair)
        else:
            if pair[1] <= pareto_front[-1][1]:
                pareto_front.append(pair)
    
    pf_X = [pair[0] for pair in pareto_front]
    pf_Y = [pair[1] for pair in pareto_front]
    x_max=max(pf_X)
    x_min=min(pf_X)
    pf_f=interp1d(pf_X, pf_Y)
    
    extended_pf=[]
    for i in np.arange(x_min,x_max,0.0001):
        extended_pf.append([i,pf_f(i)])
    pareto_region=[]
    
    for i in range(len(sorted_list)):
        distances=[]
        for j in range(len(extended_pf)):
            d=((sorted_list[i][0]-extended_pf[j][0])**2 + (sorted_list[i][1]-extended_pf[j][1])**2)**0.5
            distances.append(d)
        distances_sorted=sorted(distances)
        if distances_sorted[0] < threshold :
            pareto_region.append(sorted_list[i])
    return(pareto_region)

def get_pareto_frontiers(Xs, Ys, n, maxX=True, maxY=True):
    """
    Identifies successive Pareto fronts. The second pareto front is the Pareto front that would be obtained if the
    solutions of the (first) Pareto front were excluded, and so on.
    Arguments: Xs (array): values of first objective
               Ys (array): values of second objective
               n (int): number of successive Pareto fronts to be identified. 
    """
    data = [[Xs[i], Ys[i], i] for i in range(len(Xs))]  
    pareto_fronts = []

    for _ in range(n):
        if not data:
            break

        pareto_front = []
        remaining_data = []

        for point in data:
            x, y, _ = point
            is_dominated = False
            for other in data:
                ox, oy, _ = other
                if (maxX and ox > x or not maxX and ox < x) and (maxY and oy > y or not maxY and oy < y):
                    is_dominated = True
                    break 

            if not is_dominated:
                pareto_front.append(point)
            else:
                remaining_data.append(point)

        pareto_fronts.append(pareto_front)
        data = remaining_data 

    return pareto_fronts

In [ ]:
#load the two datasets utilized to train SGD
#SAC indicates that the dataset contains Solid, Atomic, and Compositional features
#AC indicates that the dataset contains only Atomic and Compositional features
df_SAC=pd.read_csv('./data/dataset_SAC_features.csv').set_index('material')
df_AC=pd.read_csv('./data/dataset_AC_features.csv').set_index('material')

#SGD settings
target_label=['bulk_modulus']
n_clusters=10
n_seeds=50000
n_results=5000

In [ ]:
id_job='positive_mean_shift_SAC'
write_input('./', 
            df_SAC, 
            id_job, 
            n_clusters,
            'EMM_SAMPLER',
            'normalized_positive_mean_shift',
            n_results, 
            n_seeds,
            target_label[0])
#the line below runs the realkd code, but it is commented since the output files used to obtain 
#the results described in the publication are provided
#os.system('java -jar realkd-0.7.2-jar-with-dependencies.jar '+id_job+'.json') 

In [ ]:
id_job='cJSD_SAC'
write_input('./', 
            df_SAC, 
            id_job, 
            n_clusters,
            'EMM_SAMPLER',
            'cumulative_jensen_shannon_divergence',
            n_results, 
            n_seeds,
            target_label[0])
#os.system('java -jar realkd-0.7.2-jar-with-dependencies.jar '+id_job+'.json') 

In [ ]:
id_job='cJSD_AC'
write_input('./', 
            df_AC, 
            id_job, 
            n_clusters,
            'EMM_SAMPLER',
            'cumulative_jensen_shannon_divergence',
            n_results, 
            n_seeds,
            target_label[0])
#os.system('java -jar realkd-0.7.2-jar-with-dependencies.jar '+id_job+'.json') 

In [ ]:
#analysis of SGD solutions and identification of Pareto front and Pareto region
for id_job in ['positive_mean_shift_SAC', 'cJSD_SAC','cJSD_AC']:
    file_results='./output/'+id_job+'/'+os.listdir('./output/'+id_job+'/')[0]+'/results/'+id_job+'_subgroup_analysis.json'
    df_results=analyze(file_results)
    df_results.to_csv('results_'+id_job+'.csv')
        
    pareto_front=get_pareto_frontier(df_results['coverage'], 
                                         df_results['utility'], 
                                         maxX=True, maxY=True)
    print('The Pareto front for the SG search', id_job, 'contains', len(pareto_front), 'SGs.')
    pf_indices=[pair[2] for pair in pareto_front]
    df_pf=df_results.loc[pf_indices, :]
    df_pf.sort_values(by=['coverage'],inplace=True)
    df_pf.to_csv('pf_'+id_job+'.csv')
        
    threshold=0.01
    pareto_region=get_pareto_region(df_results['coverage'], 
                                        df_results['utility'], 
                                        threshold, maxX=True, maxY=True)
    print('The Pareto region for the SG search', id_job, 'contains', len(pareto_region), 'SGs.')
    pr_indices=[pair[2] for pair in pareto_region]
    df_pr=df_results.loc[pr_indices, :]
    df_pr.sort_values(by=['coverage'],inplace=True)
    df_pr.to_csv('pr_'+id_job+'.csv')

In [ ]:
#visualization of the results
plt.rcParams.update({'font.size': 14})
#the SG index (with respect to the Pareto region, sorted according to increasing coverage) 
#can be chosen to explore the different SGs identified by the multi-objective approach
SG_index=10

for id_job in ['positive_mean_shift_SAC', 'cJSD_SAC','cJSD_AC']:
    fig,(ax1,ax2,ax3) = plt.subplots(1,3, constrained_layout=True, figsize=(10,4))
    df_results=pd.read_csv('results_'+id_job+'.csv')
    df_pf=pd.read_csv('pf_'+id_job+'.csv')
    df_pr=pd.read_csv('pr_'+id_job+'.csv')
        
    d=20
    c=['grey','darkorange','blue', 'dodgerblue','red']
    ax2.set_title('Results for SG search '+id_job)  
    ax1.scatter(df_results['coverage'],df_results['utility'],c=c[0],s=d)
    ax1.scatter(df_pr['coverage'],df_pr['utility'],c=c[3],s=d)
    ax1.scatter(df_pf['coverage'],df_pf['utility'],c=c[2],s=d)
    ax1.plot(df_pf['coverage'],df_pf['utility'],c=c[2])

    ax1.set_ylabel('$u(SG,\widetilde{P})$')
    ax1.set_xlabel('$\\frac{s(SG)}{s(\widetilde{P})}$')
    ax1.set_xlim(0,1)
    ax1.set_ylim(0,1)
    
    x=np.arange(0,1,0.01).tolist()
    Q_function=[df_results['coverage'][0]*df_results['utility'][0]/x[i] for i in range(len(x))]
    ax1.plot(x,Q_function,c=c[1],linestyle='dashed')
    ax1.scatter(df_results['coverage'][0],df_results['utility'][0],c=c[1],s=d)
    ax1.scatter(df_pr['coverage'][SG_index],df_pr['utility'][SG_index],c=c[4],s=d)
 
    ax1.text(0.1,0.9,'Pareto front',color=c[2],fontsize=12)
    ax1.text(0.1,0.85,'near Pareto front',color=c[3],fontsize=12)
    ax1.text(0.1,0.8,'SG max $Q$',color=c[1],fontsize=12)
    ax1.text(0.1,0.75,'SG with index '+str(SG_index),color=c[4],fontsize=12)
    
    bins_l=np.linspace(0.25, 1.50,num=30)
    ax2.hist(df_SAC['bulk_modulus'], color=c[0], bins=bins_l, rwidth=0.9)
    SG_max_Q=df_SAC.query(df_results['constraints'][0])
    ax2.hist(SG_max_Q['bulk_modulus'], color=c[1], bins=bins_l, rwidth=0.9)
    
    ax3.hist(df_SAC['bulk_modulus'], color=c[0], bins=bins_l, rwidth=0.9)
    SG_select_index=df_SAC.query(df_pr['constraints'][SG_index])
    ax3.hist(SG_select_index['bulk_modulus'], color=c[4], bins=bins_l, rwidth=0.9)
    
    ax2.set_xlabel('$B_0$ (eV/$\mathrm{\AA}^3$)')
    ax3.set_xlabel('$B_0$ (eV/$\mathrm{\AA}^3$)')
    ax2.set_ylabel('Counts')
    
    ax2.text(0.4,47,'entire dataset',color=c[0],fontsize=12)
    ax2.text(0.4,43,'SG max $Q$',color=c[1],fontsize=12)
    ax3.text(0.4,47,'entire dataset',color=c[0],fontsize=12)
    ax3.text(0.4,43,'SG with index '+str(SG_index),color=c[4],fontsize=12)
    
    print('Results for SG search '+id_job+':\n',
          '*constraints for SG that maximizes quality function (SG max Q) \n',df_results['constraints'][0],
          '\n *constraints for SG of the Pareto region with index',SG_index,':\n',df_pr['constraints'][SG_index])
   

In [ ]:
#analysis of SG similarity and hierarchical clustering of SGD solutions of the Pareto region
plt.rcParams.update({'font.size': 14})

for id_job in ['positive_mean_shift_SAC']:
    fig, (ax1) = plt.subplots(1,1, constrained_layout=True, figsize=(4,4))
    df_results=pd.read_csv('results_'+id_job+'.csv')
    df_pr=pd.read_csv('pr_'+id_job+'.csv')
    df_features=df_SAC.drop(['bulk_modulus'], axis=1)
    
    similarity_matrix=[]
    for i in range(len(df_pr)):
        Jaccard=[]
        for j in range(len(df_pr)):
            N1=df_features.query(df_pr.iloc[i]['constraints'])
            N2=df_features.query(df_pr.iloc[j]['constraints'])
            N1N2_combined= pd.concat([N1,N2])
            overlap=(N1N2_combined.duplicated(keep='first').sum())/len(N1N2_combined.drop_duplicates())
            Jaccard.append(overlap)
        similarity_matrix.append(Jaccard)
    ax1.set_ylabel('SG index')
    ax1.set_xlabel('SG index')
    im = ax1.imshow(similarity_matrix, vmin=0, vmax=1.0)
    fig.colorbar(im, ax=ax1, label='Jaccard index', shrink=0.6, fraction=1.2)
        
    cmap=sns.clustermap(similarity_matrix,
                   cmap="viridis", 
                   figsize=(4,4),
                   yticklabels=False,
                   xticklabels=True,
                   vmin=0,vmax=1.0)

In [ ]:
#load the dataset containing 12,096 candidate perovskites and utilized for the screening of materials
df_candidates=pd.read_csv('./data/candidates_for_screening.csv')
#randomly selecting 50 candidate perovskites
df_candidates.sample(n=50, random_state=10)

In [ ]:
job_id='cJSD_AC'
df_pf=pd.read_csv('pf_'+job_id+'.csv') 
df_results=pd.read_csv('results_'+job_id+'.csv')

#applying the SG rules associated to the SG that maximizes Q
df_candidates_selected_by_SG_max_quality=df_candidates.query(df_results['constraints'][0])
print('The SG rules associated to SG max Q select',len(df_candidates_selected_by_SG_max_quality),'materials out of 12,096 candidates.')
#randomly selecting 50 candidate perovskites from the materials that satisfy the SG rules
df_candidates_selected_by_SG_max_quality.sample(n=50, random_state=10)

In [ ]:
#applying the SG rules associated to the SGs with indicies 0, 1, 2, and 3 (with the highest utility-function values)
print('SG rules associated to SG with index 0:\n',df_pf.iloc[0]['constraints'])
print('SG rules associated to SG with index 1:\n',df_pf.iloc[1]['constraints'])
print('SG rules associated to SG with index 2:\n',df_pf.iloc[2]['constraints'])
print('SG rules associated to SG with index 3:\n', df_pf.iloc[3]['constraints'])
df_candidates_selected_by_SGs_high_utility=df_candidates.query(df_pf.iloc[0]['constraints']+'&'+df_pf.iloc[1]['constraints']+'&'+df_pf.iloc[2]['constraints'])
print('The SG rules associated to the SGs with high utility function select',len(df_candidates_selected_by_SGs_high_utility),'materials out of 12,096 candidates.')
#randomly selecting 50 candidate perovskites from the materials that satisfy the SG rules
df_candidates_selected_by_SGs_high_utility.sample(n=50, random_state=10)

In [ ]:
#comparison of bulk modulus for the materials indicated by random selection, by the SG rules identified
#with the standard approach, and by the SG rules identified with the multi-objective approach
plt.rcParams.update({'font.size': 14})

fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1, constrained_layout=True, figsize=(5,8))
#loading the file with the DFT results
df_DFT=pd.read_csv('./data/DFT_calculations.csv').set_index('material')            
df_DFT_random=df_DFT.loc[df_DFT['label'] == 'random']
df_DFT_SG_high_utility=df_DFT.loc[df_DFT['label'] == 'high utility']
df_DFT_SG_max_quality=df_DFT.loc[df_DFT['label'] == 'max']

bins_l=np.linspace(0.35, 1.70,num=30)
ax1.hist(df_SAC['bulk_modulus'], color='k', bins=bins_l, rwidth=0.9)
ax2.hist(df_DFT_random['bulk_modulus'], color='grey', bins=bins_l, rwidth=0.9)
ax4.hist(df_DFT_SG_high_utility['bulk_modulus'], color='purple', bins=bins_l, rwidth=0.9)
ax3.hist(df_DFT_SG_max_quality['bulk_modulus'], color='darkorange', bins=bins_l, rwidth=0.9)

ax4.set_xlabel('$B_0$ (eV/$\mathrm{\AA}^3$)')
ax2.set_ylabel('Counts')   
ax1.set_title('training set')
ax2.set_title('uniform sample of candidates')
ax3.set_title('candidates selected by SG rules with maximum quality')
ax4.set_title('candidates selected by SG rules with high exceptionality')
lim=15
ax1.set_ylim(0,60)
ax2.set_ylim(0,lim)
ax3.set_ylim(0,lim)
ax4.set_ylim(0,lim)

mean_data=df_SAC['bulk_modulus'].mean()
max_data=df_SAC['bulk_modulus'].max()

mean_random=df_DFT_random['bulk_modulus'].mean()
max_random=df_DFT_random['bulk_modulus'].max()

mean_SG_high_utility=df_DFT_SG_high_utility['bulk_modulus'].mean()
max_SG_high_utility=df_DFT_SG_high_utility['bulk_modulus'].max()

mean_SG_max_quality=df_DFT_SG_max_quality['bulk_modulus'].mean()
max_SG_max_quality=df_DFT_SG_max_quality['bulk_modulus'].max()

ax1.vlines(max_data,0,60,color='k',linestyle='dotted')
ax1.vlines(mean_data,0,60,color='k',linestyle='dashed')

ax2.vlines(max_random,0,15,color='k',linestyle='dotted')
ax2.vlines(mean_random,0,15,color='k',linestyle='dashed')

ax3.vlines(max_SG_max_quality,0,15,color='k',linestyle='dotted')
ax3.vlines(mean_SG_max_quality,0,15,color='k',linestyle='dashed')

ax4.vlines(max_SG_high_utility,0,15,color='k',linestyle='dotted')
ax4.vlines(mean_SG_high_utility,0,15,color='k',linestyle='dashed')